In [5]:
#basically same neural network as GA_weights_NN_reduced.ipynb but the only difference is that weights and hyperparameters are substituted by the results of GA runs

import pandas as pd
import numpy as np
import multiprocessing
from deap import base,creator,tools
import os
import subprocess
import random
import sys
import tensorflow
import pandas as pd
import numpy as np
import statistics
import tensorflow_addons as tfa 
from sklearn.utils import resample
import tensorflow as tf
import time
import keras
from tensorflow.keras import regularizers
from keras import models
from keras import layers
from keras import regularizers
from keras import metrics
from itertools import product
from keras.callbacks import EarlyStopping, ModelCheckpoint

keras.backend.clear_session()

aa_pos = {"A":0,"C":1,"D":2,"E":3,"F":4,"G":5,"H":6,"I":7,"K":8,"L":9,"M":10,"N":11,"P":12,"Q":13,"R":14,"S":15,"T":16,"V":17,"W":18,"Y":19,"-":20}
aa = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
combinations_list = ["".join(comb) for comb in product(aa, repeat=3)]

def vectorize_peptide(peptide_list):
    pept_vector = np.zeros((len(peptide_list),100,21,1))
    counter = 0
    counter_list = 0
    for peptide in peptide_list:
        if type(peptide) == float:
            continue
        if len(peptide) > 100:
            peptide = peptide[0:100]
        for i in range(len(peptide)):
            pept_vector[counter_list,i,aa_pos[peptide[i]]]=1
            counter += 1
        for j in range(counter,100):
            pept_vector[counter_list,j,aa_pos["-"]]=1
        counter_list +=1
        counter = 0
    pept_vector = np.squeeze(pept_vector)
  
    return pept_vector

def threemers(peptide_list):
    pept_vector = np.zeros((len(peptide_list),8000))
    counter_list = 0
    for peptide in peptide_list:
        counts = {aa_comb: 0 for aa_comb in combinations_list}
        for i in range(len(peptide)-2):
            window = peptide[i:i+3]
            if window in counts:
                counts[window] += 1
        pept_vector[counter_list]=list(counts.values())
        counter_list +=1
    pept_vector = pept_vector.reshape(pept_vector.shape[0],100,80)
    return pept_vector

METRICS = [keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
      keras.metrics.AUC(name='prc', curve='PR'),
          tfa.metrics.MatthewsCorrelationCoefficient(num_classes = 2,name="MCC")]

weights_vector = [6.0482598046539175, 4.861742581526707, 96, 180, 78, 0.00493846701323734, 0.6529913829209796, 0.3931546335519184, 0.7444610820039613, 0.08207272620473992, 0.0001548945392351015]
metrics_dict= []
for i in range(5):
    peptides_df = pd.read_csv("db/DB_hyperfiltered.csv")
    peptides_df.loc[peptides_df["Function"] == "Antimicrobial", "Class"] = 0 
    functions = peptides_df["Class"]
    peptides_train_60 = peptides_df.sample(frac = 0.6)
    peptides_test_20 = peptides_df.drop(peptides_train_60.index)
    peptides_validate_20 = peptides_test_20.sample(frac = 0.5)
    peptides_test_20 = peptides_test_20.drop(peptides_validate_20.index)
    all_data = vectorize_peptide(peptides_df["Sequence"])
    train_data = vectorize_peptide(peptides_train_60["Sequence"])
    test_data = vectorize_peptide(peptides_test_20["Sequence"])
    callbacks_list =[EarlyStopping(monitor='val_loss', patience=2),
         ModelCheckpoint(filepath='Antimicrobial.h5', monitor='val_MCC', save_best_only=True)] #change the model according to the class to test
    validation_data = vectorize_peptide(peptides_validate_20["Sequence"])
    all_labels = tensorflow.keras.utils.to_categorical(peptides_df["Class"].tolist(), num_classes=2)
    a = tensorflow.keras.utils.to_categorical(peptides_train_60["Class"].tolist(), num_classes=2)
    b = tensorflow.keras.utils.to_categorical(peptides_test_20["Class"].tolist(), num_classes=2)
    c = tensorflow.keras.utils.to_categorical(peptides_validate_20["Class"].tolist(), num_classes=2)
    model = models.Sequential()
    weights = {0:weights_vector[0], 1:weights_vector[1]}
    model.add(layers.Conv1D(weights_vector[3], (4,), kernel_regularizer=regularizers.L2(l2=weights_vector[5]), 
            bias_regularizer=regularizers.L2(l2=weights_vector[6]),activation='relu', input_shape=train_data[0].shape))
    model.add(layers.MaxPooling1D((2,), padding="valid"))
    model.add(layers.Conv1D(weights_vector[4], (4,), activation='relu',kernel_regularizer=regularizers.L2(l2=weights_vector[7]), bias_regularizer=regularizers.L2(l2=weights_vector[8])))
    model.add(layers.MaxPooling1D((2,)))
    model.add(layers.Dropout(weights_vector[9]))
    model.add(layers.Flatten())
    model.add(layers.Dense(weights_vector[2], activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=weights_vector[10])
    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=METRICS)
    model.summary()
    history = model.fit(train_data, a,class_weight=weights, validation_data=(validation_data,c), epochs=50, batch_size=20,use_multiprocessing=False,workers=1,verbose=1,callbacks=callbacks_list,)
    his = model.evaluate(test_data,b)
    metrics_dict.append(history)
print(metrics_dict)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 97, 180)           15300     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 48, 180)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 45, 78)            56238     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 22, 78)           0         
 1D)                                                             
                                                                 
 dropout (Dropout)           (None, 22, 78)            0         
                                                                 
 flatten (Flatten)           (None, 1716)              0

Epoch 3/50
120/120 [==============================] - 0s 4ms/step - loss: 8.5595 - categorical_accuracy: 0.8912 - prc: 0.9268 - MCC: 0.5526 - val_loss: 4.7802 - val_categorical_accuracy: 0.8988 - val_prc: 0.9325 - val_MCC: 0.6138
Epoch 4/50
120/120 [==============================] - 0s 4ms/step - loss: 4.7887 - categorical_accuracy: 0.9045 - prc: 0.9325 - MCC: 0.6132 - val_loss: 2.3837 - val_categorical_accuracy: 0.9062 - val_prc: 0.9377 - val_MCC: 0.6558
Epoch 5/50
120/120 [==============================] - 1s 5ms/step - loss: 3.0928 - categorical_accuracy: 0.9033 - prc: 0.9380 - MCC: 0.6084 - val_loss: 1.3091 - val_categorical_accuracy: 0.9125 - val_prc: 0.9447 - val_MCC: 0.6737
Epoch 6/50
120/120 [==============================] - 0s 4ms/step - loss: 2.3392 - categorical_accuracy: 0.9041 - prc: 0.9426 - MCC: 0.6132 - val_loss: 0.8731 - val_categorical_accuracy: 0.9075 - val_prc: 0.9450 - val_MCC: 0.6611
Epoch 7/50
120/120 [==============================] - 0s 3ms/step - loss: 2.0273

                                                                 
 max_pooling1d_7 (MaxPooling  (None, 22, 78)           0         
 1D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 22, 78)            0         
                                                                 
 flatten_3 (Flatten)         (None, 1716)              0         
                                                                 
 dense_6 (Dense)             (None, 96)                164832    
                                                                 
 dense_7 (Dense)             (None, 2)                 194       
                                                                 
Total params: 236,564
Trainable params: 236,564
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
120/120 [==============================] - 1s 6ms/step - lo

Epoch 12/50
120/120 [==============================] - 0s 4ms/step - loss: 1.5386 - categorical_accuracy: 0.9179 - prc: 0.9642 - MCC: 0.6848 - val_loss: 0.5703 - val_categorical_accuracy: 0.8888 - val_prc: 0.9373 - val_MCC: 0.5851
Epoch 13/50
120/120 [==============================] - 0s 4ms/step - loss: 1.4773 - categorical_accuracy: 0.9225 - prc: 0.9681 - MCC: 0.7039 - val_loss: 0.5532 - val_categorical_accuracy: 0.9000 - val_prc: 0.9468 - val_MCC: 0.6110
Epoch 14/50
120/120 [==============================] - 0s 4ms/step - loss: 1.4411 - categorical_accuracy: 0.9258 - prc: 0.9693 - MCC: 0.7185 - val_loss: 0.5407 - val_categorical_accuracy: 0.9013 - val_prc: 0.9459 - val_MCC: 0.6169
Epoch 15/50
120/120 [==============================] - 0s 4ms/step - loss: 1.4099 - categorical_accuracy: 0.9258 - prc: 0.9720 - MCC: 0.7203 - val_loss: 0.5703 - val_categorical_accuracy: 0.8913 - val_prc: 0.9340 - val_MCC: 0.5704
Epoch 16/50
25/25 [==============================] - 0s 2ms/step - loss: 0.5